<a href="https://colab.research.google.com/github/pstansfeld/MemProtMD/blob/main/MemProtMD_Association_Membrane_Protein.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/pstansfeld/MemProtMD/raw/main/mr-membrane-protein.png" height="200" align="right" style="height:240px">

##MemProtMD Association with Membrane Protein

[MemProtMD](https://doi.org/10.1016/j.str.2015.05.006) membrane protein insertion protocol adding a preformed lipid membrane around 2 provided protein structures to allow assembly of protein structures into complexes. This method is configured with a single central 'target' protein, with 8 copies of the 'search' protein. A database of [PDB](http://rcsb.org) structures can be found [here](http://memprotmd.bioch.ox.ac.uk).

The membrane-spanning regions and protein orientation is predicted using [Memembed](https://doi.org/10.1186/1471-2105-14-276); which may be downloaded from [here](https://github.com/psipred/MemEmbed).

The protein structure is converted to [Martini 3](https://www.nature.com/articles/s41592-021-01098-3) coarse-grained (CG) representation using [Martinize](https://github.com/marrink-lab/vermouth-martinize) established in a lipid membrane using [Insane](https://doi.org/10.1021/acs.jctc.5b00209) and run for 10 ns of molecular dynamics simulations using [GROMACS](https://doi.org/10.1016/j.softx.2015.06.001).

The final snapshot of the simulation is converted back to [CHARMM36m](http://mackerell.umaryland.edu/charmm_ff.shtml#gromacs) representation using [CG2AT](https://doi.org/10.1021/acs.jctc.1c00295); which can be downloaded from [here](https://github.com/pstansfeld/cg2at).

Inspiration for this Google Colab was taken from the amazing work of the [ColabFold](https://github.com/sokrypton/ColabFold) team.

Change the settings below and click **Runtime → Run all**.  You will be prompted to upload both Membrane and Soluble Proteins PDB files to simulate.

In [ ]:
#@title Define Settings
#@markdown ---
#@markdown #### Set Membrane Type:
MembraneType = "POPE:POPG" #@param ["POPC","POPE:POPG","POPE:POPG:CARDIOLIPIN"]
#@markdown ---
#@markdown #### Define Orientation of the Membrane Protein:
Nterminus = "in" #@param ["in","out"]
#@markdown ---
#@markdown #### Set number of Repeats
Repeats = 5 #@param {type:"slider", min:1, max:25, step:1}
#@markdown ---
#@markdown #### Define Distance between Proteins:
Distance = 15 #@param {type:"slider", min:0, max:25, step:1}
#@markdown ---
#@markdown #### Set Box Dimensions:
Box_Width = 12 #@param {type:"slider", min:8, max:30, step:1}
Box_Length = 30 #@param {type:"slider", min:12, max:50, step:1}
#@markdown ---

In [ ]:
#@title Upload PDB coordinate files
import os
import sys

cube = [Box_Width,Box_Width,Box_Length]

if MembraneType == "POPC":
  lipid = '-l POPC:1'
elif MembraneType == "POPE:POPG":
  lipid = '-l POPE:7 -l POPG:3'
elif MembraneType == "POPE:POPG:CARDIOLIPIN":
  lipid = '-l POPE:7 -l POPG:2 -l CARD:1'

!python3 -m pip install py3dmol
!python3 -m pip install colorama

import py3Dmol
from colorama import Fore
from google.colab import files
sys.path.append('/usr/local/lib/python3.7/site-packages/')

os.chdir('/content/')
print(Fore.BLUE + "\nUpload Membrane Protein:\n")
upload1 = files.upload()
print(Fore.BLUE + "\nUpload Soluble Protein:\n")
upload2 = files.upload()
mview = py3Dmol.view(width=800,height=400)  
filename1 = next(iter(upload1))
name1 = os.path.splitext(filename1)[0]
filename2 = next(iter(upload2))
name2 = os.path.splitext(filename2)[0]
name = name1 + '-' + name2
working_dir = '/content/' + name1 + '-' + name2 +'/'
os.makedirs(working_dir, exist_ok=True)
os.rename(filename1, working_dir + filename1)
os.chdir(working_dir)
mol1 = open(working_dir + filename1, 'r').read()
mview.addModel(mol1,'pdb')
mview.setStyle({'cartoon':{'color':'spectrum'}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

os.chdir('/content/')
mview = py3Dmol.view(width=800,height=400)  
os.rename(filename2, working_dir + filename2)
os.chdir(working_dir)
mol2 = open(working_dir + filename2, 'r').read()
mview.addModel(mol2,'pdb')
mview.setStyle({'cartoon':{'color':'spectrum'}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

In [16]:
#@title Install dependencies
%%capture
os.chdir('/content/')
if not os.path.isdir("/content/memembed/"):
  !apt-get update -y
  !apt-get install dssp
  !git clone https://github.com/timnugent/memembed
  %cd memembed
  !make
  %cd ../
  !python3 -m pip install pdb2pqr
  !python3 -m pip install vermouth
  !python3 -m pip install GromacsWrapper
  !python3 -m pip install MDAnalysis 
  !git clone https://github.com/pstansfeld/cg2at
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/martini_v300.zip
  !unzip -o martini_v300.zip
  !wget 'http://cgmartini.nl/images/applications/water/water.gro'
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/insane3.py
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/gromacs.zip
  !unzip -o gromacs.zip
  !ln -s /content/content/gromacs/bin/gmx /usr/bin/gmx

import gromacs
import MDAnalysis
import glob
import shutil

In [ ]:
#@title Predict Membrane Orientations
os.chdir(working_dir)
os.system('/content/memembed/bin/memembed -n ' + Nterminus + ' -o memembed1.pdb ' + filename1) 
mview = py3Dmol.view(width=800,height=400)  
mol1 = open(working_dir + '/memembed1.pdb', 'r').read()
mview.addModel(mol1)
mview.setStyle({'cartoon':{'color':'spectrum'}})
mview.setStyle({'resn':'DUM'},{'sphere':{}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()

In [18]:
#@title Set-up Coarse-Grained Membrane Protein Systems
%%capture
os.chdir(working_dir)

for file in glob.glob(r'/content/martini_v300/martini*.itp'):
    print(file)
    shutil.copy(file, working_dir)

gromacs.make_ndx(f='memembed1.pdb', o='index1.ndx', input=('del 0', 'del 1-100','rDUM','q'),backup=False)
gromacs.editconf(f='memembed1.pdb',o='centered1.pdb',n='index1.ndx',c=True,box=cube,input=(0,0),backup=False)

u = MDAnalysis.Universe('centered1.pdb')
x = round(u.dimensions[0]/10)
y = round(u.dimensions[1]/10)
z = round(u.dimensions[2]/10)

gromacs.confrms(f2='memembed1.pdb', f1='centered1.pdb', one=True, o='aligned1.gro', input=(3,3),backup=False)
gromacs.editconf(f='aligned1.gro', o='protein1.pdb',label='A',resnr=1,n='index1.ndx',input=(0,0),backup=False)

v = MDAnalysis.Universe('aligned1.gro')
dum = v.select_atoms('resname DUM')

dm = (z/2) - (round(dum.center_of_mass()[2]/10))

with open('protein1.pdb', 'r') as file :
  filedata = file.read()
filedata = filedata.replace('HSE', 'HIS')
filedata = filedata.replace('HSD', 'HIS')
filedata = filedata.replace('MSE', 'MET')
filedata = filedata.replace(' SE ', ' SD ')
with open('protein1.pdb', 'w') as file:
  file.write(filedata)
with open('em.mdp','w') as em:
            em.write('integrator = steep\nnsteps = 5000\nemtol = 100\nemstep = 0.001')
with open('topol.top','w') as top:
            top.write('#include "martini_v3.0.0.itp"\n#include "martini_v3.0.0_ions_v1.itp"\n#include "martini_v3.0.0_solvents_v1.itp"\n#include "martini_v3.0.0_phospholipids_v1.itp"\n#include "protein1-cg.itp"\n[ system ]\n[ molecules ]\nProteinA 1')

!martinize2 -f protein1.pdb -ff martini3001 -x protein1-cg.pdb -o protein1-cg.top -elastic -merge A -maxwarn 100000
!sed -e 's/^molecule.*/ProteinA 1/g' molecule_0.itp >  protein1-cg.itp
file_object = open('protein1-cg.itp', 'a')
file_object.write('#ifdef POSRES\n#include "protein1-posre.itp"\n#endif\n')
file_object.close()
gromacs.genrestr(f='protein1-cg.pdb',o='protein1-posre.itp',input=(1,1))

gromacs.editconf(f='protein1-cg.pdb', o='protein1-box.pdb', c=True, box=cube)
gromacs.grompp(f='em.mdp',o='em.tpr',c='protein1-box.pdb',maxwarn='-1',backup=False,v=True)
gromacs.mdrun(deffnm='em', c='protein1-em.pdb',backup=False)

gromacs.make_ndx(f=filename2, o='index2.ndx', input=('del 0', 'del 1-100','rDUM','q'),backup=False)
gromacs.editconf(f=filename2,o='protein2.pdb',n='index2.ndx',c=True,box=cube,input=(0,0),backup=False,label='A',resnr=1)

with open('protein2.pdb', 'r') as file :
  filedata = file.read()
filedata = filedata.replace('HSE', 'HIS')
filedata = filedata.replace('HSD', 'HIS')
filedata = filedata.replace('MSE', 'MET')
filedata = filedata.replace(' SE ', ' SD ')
with open('protein2.pdb', 'w') as file:
  file.write(filedata)
with open('topol.top','w') as top:
            top.write('#include "martini_v3.0.0.itp"\n#include "martini_v3.0.0_ions_v1.itp"\n#include "martini_v3.0.0_solvents_v1.itp"\n#include "martini_v3.0.0_phospholipids_v1.itp"\n#include "protein2-cg.itp"\n[ system ]\n[ molecules ]\nProteinB 1')

!martinize2 -f protein2.pdb -dssp mkdssp -ff martini3001 -x protein2-cg.pdb -o protein2-cg.top -elastic -merge A -maxwarn 100000
!sed -e 's/^molecule.*/ProteinB 1/g' molecule_0.itp >  protein2-cg.itp
file_object = open('protein2-cg.itp', 'a')
file_object.write('#ifdef POSRES\n#include "protein2-posre.itp"\n#endif\n')
file_object.close()
gromacs.genrestr(f='protein2-cg.pdb',o='protein2-posre.itp',input=(1,1))

gromacs.editconf(f='protein2-cg.pdb', o='protein2-box.pdb', c=True, box=cube)
gromacs.grompp(f='em.mdp',o='em.tpr',c='protein2-box.pdb',maxwarn='-1',backup=False,v=True)
gromacs.mdrun(deffnm='em', c='protein2-em.pdb',backup=False)

gromacs.editconf(f='protein2-em.pdb',o='protein2-em.pdb',translate=[0,0,-Distance])

for rep in range(1,Repeats+1):
  rep=str(rep)
  os.chdir(working_dir)
  os.system('python /content/insane3.py ' + lipid + ' -o protein1-system.gro -p topol.top -f protein1-em.pdb -center -ring -x %s -y %s -z %s -dm %s' % (x, y, z, dm))
  !wget 'http://cgmartini.nl/images/applications/water/water.gro'
  u1 = MDAnalysis.Universe('protein1-system.gro')
  x = round(u.dimensions[0]/10)
  y = round(u.dimensions[1]/10)
  z = round(u.dimensions[2]/10)
  u2 = MDAnalysis.Universe("protein2-em.pdb")
  multi = MDAnalysis.Merge(u2.atoms, u1.atoms)
  multi.atoms.write("protein12.pdb")

  replacements = {'Protein        1\n':'ProteinB       1\nProteinA       1\n','#include "protein-cg.itp"':'#include "protein1-cg.itp"\n#include "protein2-cg.itp"\n','NA+':'NA', 'CL-':'CL', '#include "martini_v3.itp"':'#include "martini_v3.0.0.itp"\n#include "martini_v3.0.0_ions_v1.itp"\n#include "martini_v3.0.0_solvents_v1.itp"\n#include "martini_v3.0.0_phospholipids_v1.itp"\n'}
  lines = []
  with open('topol.top') as infile:
      for line in infile:
          for src, target in replacements.items():
              line = line.replace(src, target)
          lines.append(line)
  with open('topol.top', 'w') as outfile:
      for line in lines:
          outfile.write(line)
  
  os.system('cp topol.top w.top')

  gromacs.solvate(cp='protein12.pdb',o='prot+lipid+sol.pdb',cs='water.gro',radius=0.21,backup=False,box=cube)
  v = MDAnalysis.Universe('prot+lipid+sol.pdb')
  wat = v.select_atoms('name W')
  w = wat.atoms.n_atoms

  file_object = open('w.top', 'a')
  file_object.write('\nW ' + str(w) )
  file_object.close()

  gromacs.grompp(f='em.mdp',o='ion.tpr',c='prot+lipid+sol.pdb',maxwarn='-1',backup=False,v=True,p='w.top')
  gromacs.genion(s='ion.tpr',o='CG-system.gro',neutral=True,conc=0.15,backup=False,input='W')

  v = MDAnalysis.Universe('CG-system.gro')
  wat = v.select_atoms('name W')
  sod = v.select_atoms('name NA')
  chl = v.select_atoms('name CL')
  w = wat.atoms.n_atoms
  na = sod.atoms.n_atoms
  cl = chl.atoms.n_atoms

  file_object = open('topol.top', 'a')
  file_object.write('\nW ' + str(w) + '\nNA ' + str(na) + '\nCL ' + str(cl) )
  file_object.close()

  os.system('cp topol.top topol'+rep+'.top')

  gromacs.grompp(f='em.mdp',o='em'+ rep +'.tpr',c='CG-system.gro',maxwarn='-1',backup=False,v=True)
  gromacs.mdrun(deffnm='em'+ rep, c='CG-system'+ rep +'.pdb',backup=False)
  gromacs.trjconv(f='CG-system'+ rep +'.pdb', o='CG-system'+ rep +'.pdb', pbc='mol', s='em'+ rep +'.tpr', conect=True, input=[0,0],backup=False)

  os.makedirs('EQ'+rep, exist_ok=True)
  gromacs.make_ndx(f='CG-system'+ rep +'.pdb', o='index'+ rep +'.ndx', input=('del 0', 'del 1-40', '0|rPOP*','1&!0','!1','del 1','name 1 Lipid','name 2 SOL_ION','q'),backup=False)
  with open('cgequil.mdp','w') as md:
            md.write('define = -DPOSRES\nintegrator = md\ntinit = 0.0\ndt = 0.02\nnsteps = 10000\nnstxout = 0\nnstvout = 0\nnstfout = 0\nnstlog = 50000\nnstenergy = 50000\nnstxout-compressed = 50000\ncompressed-x-precision = 10000\nnstlist  = 10\nns_type  = grid\npbc   = xyz\ncoulombtype  = Reaction_field\nrcoulomb_switch = 0.0\nrcoulomb  = 1.1\nepsilon_r  = 15\nvdw_type  = cutoff\nrvdw_switch  = 0.9\nrvdw   = 1.1\ncutoff-scheme = verlet\ncoulomb-modifier = Potential-shift\nvdw-modifier  = Potential-shift\nepsilon_rf  = 0\nverlet-buffer-tolerance = 0.005\ntcoupl  = v-rescale\ntc-grps  = PROTEIN LIPID SOL_ION\ntau_t  = 1.0 1.0 1.0\nref_t  = 310 310 310\nPcoupl  = berendsen\nPcoupltype  = semiisotropic\ntau_p  = 12.0\ncompressibility = 3e-4 3e-4\nref_p  = 1.0 1.0\ngen_vel  = yes\ngen_temp  = 310\ngen_seed  = -1\nconstraints  = none\nconstraint_algorithm = Lincs\ncontinuation  = no\nrefcoord_scaling = com')
  gromacs.grompp(f='cgequil.mdp',o='EQ'+ rep +'/eq',r='CG-system'+ rep +'.pdb',c='CG-system'+ rep +'.pdb',maxwarn=-1, n='index'+ rep +'.ndx',backup=False)  
  os.chdir(working_dir + '/EQ' + rep)
  gromacs.mdrun(deffnm='eq',backup=False, v=True)
  os.chdir(working_dir)
  os.makedirs('MD'+rep, exist_ok=True)
  with open('cgmd.mdp','w') as md:
            md.write('integrator = md\ntinit = 0.0\ndt = 0.02\nnsteps = 250000000\nnstxout = 0\nnstvout = 0\nnstfout = 0\nnstlog = 50000\nnstenergy = 50000\nnstxout-compressed = 50000\ncompressed-x-precision = 10000\nnstlist  = 10\nns_type  = grid\npbc   = xyz\ncoulombtype  = Reaction_field\nrcoulomb_switch = 0.0\nrcoulomb  = 1.1\nepsilon_r  = 15\nvdw_type  = cutoff\nrvdw_switch  = 0.9\nrvdw   = 1.1\ncutoff-scheme = verlet\ncoulomb-modifier = Potential-shift\nvdw-modifier  = Potential-shift\nepsilon_rf  = 0\nverlet-buffer-tolerance = 0.005\ntcoupl  = v-rescale\ntc-grps  = PROTEIN LIPID SOL_ION\ntau_t  = 1.0 1.0 1.0\nref_t  = 310 310 310\nPcoupl  = berendsen\nPcoupltype  = semiisotropic\ntau_p  = 12.0\ncompressibility = 3e-4 3e-4\nref_p  = 1.0 1.0\ngen_vel  = yes\ngen_temp  = 310\ngen_seed  = -1\nconstraints  = none\nconstraint_algorithm = Lincs\ncontinuation  = no\n')
  gromacs.grompp(f='cgmd.mdp',o='MD'+ rep +'/md',c='EQ'+ rep +'/eq.gro',maxwarn=-1, n='index'+ rep +'.ndx',backup=False)  
  

In [ ]:
#@title Visualise Final systems
for rep in range(1,Repeats+1):
  rep=str(rep)
  print('\nRepeat ' + rep + '\n')
  mview = py3Dmol.view(width=800,height=400)  
  mol1 = open(working_dir + 'EQ'+rep +'/eq.gro', 'r').read()
  mview.addModel(mol1,'gro')
  mview.setStyle({'cartoon':{'color':'spectrum'}})
  mview.setStyle({'atom':'PO1','atom':'PO2','atom':'PO3','atom':'PO4'},{'sphere':{}})
  mview.setStyle({'atom':'BB'},{'sphere':{}})
  mview.setBackgroundColor('0xffffff')
  mview.zoomTo()
  mview.show()
  for file in glob.glob(r'#*'):
      os.remove(file)
    
os.chdir(working_dir)
for file in glob.glob(r'#*'):
    os.remove(file)

In [20]:
#@title Download Zip
os.chdir(working_dir)
shutil.rmtree('MD', ignore_errors=True)
try:
    os.remove('em*.trr')
except OSError:
    pass
try:
    os.remove('em*.tpr')
except OSError:
    pass
try:
    os.remove('em*.log')
except OSError:
    pass
try:
    os.remove('protein.pdb')
except OSError:
    pass
try:
    os.remove('molecule_0.itp')
except OSError:
    pass
try:
    os.remove('mdout.mdp')
except OSError:
    pass
try:
    os.remove('aligned.gro')
except OSError:
    pass
try:
    os.remove('protein-cg.top')
except OSError:
    pass
os.system('zip -r /content/' + name + '.zip ' + working_dir )
files.download('/content/' + name + '.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>